<a href="https://colab.research.google.com/github/VerandaC/YourFarmerLlama/blob/main/Modelo_de_predicion_de_agricultura__llama_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **importamos el dataset**

In [49]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("calvom/agricultura")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/calvom/agricultura/versions/3


# inspecionamos las **columnas**

In [50]:
print(df.columns)

Index(['ID Provincia', 'Provincia', 'ID Departamento', 'Departamento',
       'Id Cultivo', 'Cultivo', 'ID Campai? 1/2a', 'Campana',
       'Temperatura (Ce)', 'Sup. Sembrada (Ha)', 'Sup. Cosechada (Ha)',
       'Produccii? 1/2n (Tn)', 'Rendimiento (Kg/Ha)', 'Calidad'],
      dtype='object')


In [51]:
df

,ID Provincia,Provincia,ID Departamento,Departamento,Id Cultivo,Cultivo,ID Campai? 1/2a,Campana,Temperatura (Ce),Sup. Sembrada (Ha),Sup. Cosechada (Ha),Produccii? 1/2n (Tn),Rendimiento (Kg/Ha),Calidad
0,6,BUENOS AIRES,854,25 DE MAYO,1,Ajo,1,1969/70,23,3.00,3.00,10,3.333,Alta
1,6,BUENOS AIRES,854,25 DE MAYO,1,Ajo,2,1970/71,21,1.00,1.00,3,3.000,Promedio
2,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,1,1969/70,30,15.00,15.00,82,5.467,Promedio
3,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,2,1970/71,31,10.00,10.00,55,5.500,Alta
4,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,3,1971/72,26,8.00,8.00,44,5.500,Alta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132764,54,MISIONES,112,SAN PEDRO,31,Yerba mate,40,2008/09,31,5.50,4.80,23.76,4.950,Promedio
132765,54,MISIONES,112,SAN PEDRO,31,Yerba mate,41,2009/10,31,5.50,4.80,24.48,5.100,Promedio
132766,54,MISIONES,112,SAN PEDRO,31,Yerba mate,43,2011/12,31,5.85,5.15,23.175,4.500,Baja
132767,54,MISIONES,112,SAN PEDRO,31,Yerba mate,45,2013/14,31,5.85,4.85,26.19,5.400,Baja


# importamos las **librerias**

In [ ]:
from IPython import get_ipython
from IPython.display import display
import kagglehub
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
import unidecode

# Descarga la última versión del dataset
dataset_path = kagglehub.dataset_download("calvom/agricultura")

# Imprime la ruta del conjunto de datos descargado
print("Ruta del conjunto de datos descargado:", dataset_path)

# Lista los archivos en el directorio del conjunto de datos descargado
print("Archivos en el directorio del conjunto de datos:", os.listdir(dataset_path))

# Verifica si 'Agricultura.csv' está presente en el directorio del conjunto de datos
csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]

if csv_files:
    # Si se encuentran archivos csv, usar el primero
    csv_file_path = os.path.join(dataset_path, csv_files[0])

    # Leer el archivo CSV usando la ruta correcta y especificando la codificación
    df = pd.read_csv(csv_file_path, encoding='latin-1')  # O encoding='utf-8'

    # Quitar las tildes en los nombres de las columnas
    df.columns = [unidecode.unidecode(col) for col in df.columns]

    # ----> Limpieza de la columna 'Produccii? 1/2n (Tn)'
    df['Produccii? 1/2n (Tn)'] = df['Produccii? 1/2n (Tn)'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    df['Produccii? 1/2n (Tn)'] = pd.to_numeric(df['Produccii? 1/2n (Tn)'], errors='coerce')

    # ----> Eliminar filas con NaN en 'Produccii? 1/2n (Tn)'
    df = df.dropna(subset=['Produccii? 1/2n (Tn)'])

    # Seleccionar variables predictoras y variable objetivo
    X = df[['Sup. Cosechada (Ha)', 'Rendimiento (Kg/Ha)']]
    y = df['Produccii? 1/2n (Tn)']

    # Entrenar el modelo
    model = LinearRegression()
    model.fit(X, y)

    # Función para la entrada de datos
    def obtener_datos_usuario():
        superficie = float(input("Ingrese la superficie cosechada: "))
        rendimiento = float(input("Ingrese el rendimiento: "))
        return superficie, rendimiento